In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import pickle

# Load the test data
test_data = pd.read_csv("sample_test_data.csv")##YOU CAN PROVIDE UR TEST DATASET HERE

# Drop unnecessary columns (similar to what you did in the training file)
column_to_drop = 'datasetId'
test_data.drop(column_to_drop, axis=1, inplace=True)

# Load the trained model
loaded_model = load_model('model.h5')

# Load the scaler parameters
scaler_params_path = 'scaler_params.pkl'
with open(scaler_params_path, 'rb') as scaler_file:
    scaler_params = pickle.load(scaler_file)

# Set up the scaler
standard_scaler = MinMaxScaler()
standard_scaler.min_, standard_scaler.scale_ = scaler_params['min_'], scaler_params['scale_']

# Preprocess the test input data
test_data_numeric = test_data.select_dtypes(exclude=['object'])
columns_to_scale = ['MEAN_RR', 'MEDIAN_RR', 'LF_NU', 'HF_NU', 'HF_LF', 'SDRR_RMSSD_REL_RR', 'HF_PCT', 'HF', 'SDSD_REL_RR', 'RMSSD_REL_RR', 'higuci', 'LF_HF', 'VLF', 'TP', 'sampen', 'SKEW', 'SKEW_REL_RR']
test_data_numeric[columns_to_scale] = standard_scaler.transform(test_data_numeric[columns_to_scale])

# Specify the features for prediction
X_test = test_data_numeric[columns_to_scale]

# Reshape the input data to match the model's input shape
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# Make predictions on the test data
predictions_test = loaded_model.predict(X_test_reshaped)

# Add 'uuid' and corresponding predictions to a new DataFrame
predictions_df = pd.DataFrame({'uuid': test_data['uuid'], 'predictions': predictions_test.flatten()})

# Print the first few rows of the predictions DataFrame
print("Predictions on Test Data:")
print(predictions_df.head(10))


c:\Users\Dell\anaconda3_original\envs\py310\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 1s 745ms/step
Predictions on Test Data:
                                   uuid  predictions
0  1ae30e0b-098e-46fc-a897-0a6661f26370    75.252983
1  428b41b3-9461-4c79-ab4e-d03b122b2553    81.012993
2  88f82ac7-02dd-447e-a289-22e8e22432c2    63.571381
3  1d09b18f-d82f-4c1a-bb2d-71fda6fea837    66.546227
4  a6302640-f70a-4a3a-ad36-a8c3d5df9400    65.667717
5  3f6508be-4b0a-4008-b701-49d8c2d5dd43    57.519295
6  a07d84c8-fc44-45ef-bb85-f06f06b70e9f    73.164726
7  f4a449db-a7ff-437b-852b-821a6e965f2f    63.505524
8  94364ef1-12e2-4ddd-9f35-99e270547849    57.943321
9  231d34f5-1028-4f2e-8e1d-00d086b0c218    71.137177


In [5]:
predictions_df.to_csv('results.csv', index=False)